Contexto

In [ ]:
#Instalação das bibliotecas
!pip install --q crewai
!pip install --q 'crewai[tools]'
!pip install --q langchain_groq
!pip install --q langchain_chatOpenAI
!pip install --q dotenv
!pip install --q textwrap3

ERROR: Could not find a version that satisfies the requirement langchain_chatOpenAI (from versions: none)
ERROR: No matching distribution found for langchain_chatOpenAI
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
#Importação das bibliotecas
import os
from crewai_tools.tools.scrape_website_tool.scrape_website_tool import ScrapeWebsiteTool
from dotenv import load_dotenv
from datetime import datetime
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from textwrap import dedent

In [ ]:
#Obtenção das variáveis de ambiente do aruivo .env
load_dotenv()

False

In [ ]:
#Configuração das chaves de API
open_api_key = os.getenv("OPEN_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

In [ ]:
#Configuração das LLMs (GENAIs)
llm_gpt35 = ChatOpenAI(model="gpt-3.5-turbo", api_key=openai_api_key)
llm_gpt4o = ChatOpenAI(model="gpt-4o", api_key=openai_api_key)
llm_llama3 = ChatGroq(model="llhama3-70b-8192", api_key=groq_apy_key)

NameError: name 'openai_api_key' is not defined

In [ ]:
#Configuração do Scrapweb
def criar_scrape_tools(urls):
    tools = []
    for url in urls:
        tool = ScrapeWebsiteTool(website_url=url)
        tools.append(tool)
    return tools

In [ ]:
#Criação dos agentes
orquestrator = Agent(
    role="Gerente dos agentes",
    goal="Gerenciar o conteúdo gerado pelos agentes pesquisadores sobre {topic}.",
    backstory="irá trabalhar na orquestração (gerenciamento) de {n} agentes para organizar os resultados sobre o {topic}.",
    verbose=True,
    max_rpm=1,
    llm=llm_gpt4o
)
pesquisador = Agent(
    role="Pesquisar",
    goal="Pesquisar as tendências sobre {topic}.",
    backstory="é uma pesquisador experiente, que busca as tendências sobre {topic}.",
    verbose=True,
    tools=urls,
    llm=llm_gpt4o
)
publicador = Agent(
    role="Publicador",
    goal="Escrever as tendências sobre {topic}.",
    backstory="é um escritor experiente, que escreve o resultado das buscas dos pesquisadores em português do Brasil formal e no estilo científico sobre o {topic}.",
    verbose=True,
    tools=urls,
    llm=llm_gpt35
)
Instagramador = Agent(
    role="Instagramador",
    goal="Gerar imagens e vídeos impactantes sobre {topic}.",
    backstory="é um instagramador experiente, que gera imagens e vídeos atraentes sobre o {topic}.",
    verbose=True,
    tools=urls,
    llm=llm_llama3
)

In [ ]:
#Definição das atividades
revisa_task = Task(
    description=(
        "1. Encontrar as últimas tendências sobre {topic}\n"
        "2. Focar no público que se interessa sobre Inteligência Artificial\n"
        "3. Construir uma introdução, pontos chaves e uma chamada envolvente\n"
        "4. Incluir as palavras-chaves e fontes."
    ),
    expected_output="Uma postagem sobre {topic} com {n} posts",
    agent=orquestrador,
    allow_delegation=False,
    verbose=True
)
pesquisa_task = Task(
    description=("Pesquisar as últimas tendêcias sobre {topic}."),
        expected_output="Encontrar as notícias mais relevantes sobre {topic} na área de Inteligência Artificial",
        agent=pesquisador,
        allow_delegation=False,
        verbose=True
)
escreve_task = Task(
    description=("Pesquisar as últimas tendêcias sobre {topic}."),
        expected_output="Escrever {n} postagens sobre {topic}",
        agent=publicador,
        allow_delegation=False,
        verbose=True
)
desenha_task = Task(
    description=("Pesquisar as últimas tendêcias sobre {topic}."),
        expected_output="Escrever {n} postagens sobre {topic}",
        agent=Instagramador,
        allow_delegation=False,
        verbose=True
)

In [ ]:
#Definição das URLs
#URLs
urls = [
    'https://www.tecmundo.com.br'
    'https://www.techtudo.com.br'
]

In [ ]:
#Definição da saída
crew = Crew(
    agents = [pesquisador, publicador, instgramador, orquestrador],
    tasks = [pesquisa_task, escreve_task, desenha_task, revisa_task],
    process = Process.sequential,
    verbose = True,
    memory = True,
)
criar_scrape_tools(urls)
resultado = crew.kickoff(inputs={
    'topic':'Multiagentes',
    'n':1
})
print(resultado)